# Deep copy

---
**Requirements:**

- [Get Started](./Get_started.ipynb)
- [Data Management](./Data_management.ipynb)
- [Atomic Operations](./Atomic_operations.ipynb)

---

Complex data structures, including struct and classes in C or derived datatypes with pointers and allocatable in Fortran, are frequent. Ways to managed them include:

- using CUDA unified memory with the compilation flag `-gpu:managed`, but the cost of memory allocation will be higher and it will apply to all allocatable variables
- flatten the derived datatypes by using temporary variables and then perform data transfers on the temporary variables
- using deep copy.

Two ways are possible to manage deep copy:

- top-down deep copy with an implicit attach behavior
- bottom-up deep copy with an explicit attach behavior

## Top-down deep copy

In order to implement the top-down deep copy, we should copy to the device the base structure first and then the children structures.
For each children transfer, the compiler's implementation will check if the pointers to the children (they are transferred with the parent structure) are present. If they are, an implicit attach behavior is performed and the parents on the device will point toward the children that are newly put on the device.

Please note that it is not mandatory to transfer all the children structure, only the ones that calculations on the device require.

### Syntax

```C
typedef class{
    float *vx, *vy, *vz;
}velocity;

...
velocity U;
U.vx = (float*) malloc(sizeX*sizeof(float));;
U.vy = (float*) malloc(sizeY*sizeof(float));
U.vz = (float*) malloc(sizeZ*sizeof(float));
...

#pragma acc enter data copy(U)
#pragma acc enter data copy(U.vx[0:sizeX], U.vy[0:sizeY], U.vz[0:sizeZ])

// A humonguous calculation
```

### Example

In this example we store 2 arrays in a structure/derived type and use a deep copy to make them available on the GPU.

Example stored in: `../../examples/C/Deep_copy_example.c`

In [ ]:
%%idrrun -a
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

typedef struct
{
    double* s;
    double* c;
} Array;

Array* allocate_array(size_t size);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->s = (double*) malloc(size*sizeof(double));
    arr->c = (double*) malloc(size*sizeof(double));
    return arr;
}

int main(void)
{
    int size=1e5;
    double sum[size];

    Array* vec;
    vec = allocate_array(size);

    #pragma acc data create(vec, vec->s[:size], vec->c[:size]) copyout(sum)
    {
    #pragma acc parallel
    {
        #pragma acc loop
        for (int i=0; i<size;++i)
        {
            vec->s[i] = sin(i*M_PI/size)*sin(i*M_PI/size);
            vec->c[i] = cos(i*M_PI/size)*cos(i*M_PI/size);
        }    
    }
    #pragma acc parallel
    {
        #pragma acc loop
        for (int i=1; i<size ; ++i)
            sum[i] = vec->s[i] + vec->c[size - i]; 
    }
    }// end of structured data region
    printf("sum[42] = %f\n", sum[42]);
}


### Exercise

In this exercise, we determine the radial distribution function (RDF) for an ensemble of particles that is read from a file. The position of the particles can be use as a demonstration on the implementation of the deep copy.
You can run this example at the end of the next exercise to check the structure of the box at the end of the simulation.

First you need to copy some files:

In [ ]:
%%bash
cp ../../examples/dyn.xyz .

You need to pass `--cliopts "0.5 15.5"` to idrrun.

Example stored in: `../../examples/C/Deep_copy_exercise.c`

In [ ]:
%%idrrun -a --cliopts "0.5 15.5"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
# include <math.h>
#include <float.h>

typedef struct
{
    size_t size;
    double* data;
} Array;

typedef struct
{
    Array* x;
    Array* y;
    Array* z;
} Coordinates;

typedef struct
{
    size_t natoms;    // number of atoms
    double lx;        // box length in each dimension
    double ly;         
    double lz;         
} Config;

Array* allocate_array(size_t size);
void free_array(Array* arr);
Coordinates* read_coords(char* filepath, Config *config);
void free_coords(Coordinates* coords);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->size = size;
    arr->data = (double*) malloc(size*sizeof(double));
    return arr;
}

Coordinates* allocate_coords(size_t size)
{
    Coordinates* coords = (Coordinates*) malloc(sizeof(Coordinates));
    coords->x  = allocate_array(size);
    coords->y  = allocate_array(size);
    coords->z  = allocate_array(size);
    return coords;
 }

void free_array(Array* arr)
{
    free(arr->data);
    free(arr);
}

void free_coords(Coordinates* coords)
{
    free_array(coords->x);
    free_array(coords->y);
    free_array(coords->z);
    free(coords);
}

Coordinates* read_coords(char* filepath, Config* conf)
{
    FILE* fptr = fopen(filepath, "r");
    char* line = NULL;
    size_t len = 0;
    size_t i = 0;
    char n[20], x[20], y[20], z[20], vx[20], vy[20], vz[20];
    
    if (fptr == NULL)
        exit(EXIT_FAILURE);

    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->natoms = atoi(n);
    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->lx = atof(n);
    conf->ly = atof(n);
    conf->lz = atof(n);
    printf("Number of atoms in %s file: %d\n", filepath, conf->natoms);
    
    Coordinates* coords = allocate_coords(conf->natoms);
    
    while (i < conf->natoms)
    {
        getline(&line, &len, fptr);
        sscanf(line, "%s %s %s %s %s %s %s", n, x, y, z, vx, vy, vz);
        coords->x->data[i] = atof(x);
        coords->y->data[i] = atof(y);
        coords->z->data[i] = atof(z);
	++i;
    }
    fclose(fptr);
    return coords;
}

int main(int argc, char** argv)
{
    double deltaR, rCutOff;
    FILE* fPtr;
    char* input;
    double xij, yij, zij, rij;
    int d;
    
    if (argc < 3 || argc > 4) 
    {
        fprintf(stderr, "%s", "ERROR: Wrong number of parameters.\n");
        fprintf(stderr, "%s", "ERROR: The program requires at least two parameters:\n");
        fprintf(stderr, "%s", "ERROR: deltaR, the length of each bin, and \n");
        fprintf(stderr, "%s", "ERROR: rCutoff, the total length (rcut < box_length/2).\n");
        fprintf(stderr, "%s", "ERROR: Usage example: ./rdf 0.5 15.5 [input]\n");
        exit(EXIT_FAILURE);
    }
    else
    {
        deltaR = atof(argv[1]);
        rCutOff = atof(argv[2]);
        if (argc == 4)
            input = argv[3];
        else
            input = "./dyn.xyz";
    }
    
    int maxbin = rCutOff/deltaR + 1;
    int* hist = (int*) malloc(maxbin*sizeof(int));
    double* gr = (double*) malloc(maxbin*sizeof(double));
    
    for (size_t i=0; i<maxbin; ++i)
        hist[i] = 0;
    
    Config* conf = (Config*) malloc(sizeof(Config));
    Coordinates* coords = read_coords(input, conf);  

    #pragma acc parallel loop present(conf,coords,coords->x,coords->y,coords->z)\
                              present(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    for (int j = 0; j < conf->natoms; ++j)
        #pragma acc loop private(xij,yij,zij,rij,d)
        for (int i = 0; i < conf->natoms; ++i)
            if (i != j)		
            {
	        xij = coords->x->data[j]-coords->x->data[i];
                yij = coords->y->data[j]-coords->y->data[i];
                zij = coords->z->data[j]-coords->z->data[i];
                xij -= floor(xij/conf->lx + 0.5) *conf->lx;
                yij -= floor(yij/conf->ly + 0.5) *conf->ly;
                zij -= floor(zij/conf->lz + 0.5) *conf->lz;
		rij = xij*xij + yij*yij + zij*zij;
                d = (int) (sqrt(rij)/deltaR);
                if (d < maxbin)
                    #pragma acc atomic update
                    ++hist[d];
        }
    
    double rho = ((double) conf->natoms) / ((double)(conf->lx * conf->ly * conf->lz)) * 4.0 / 3.0 * acos(-1.0);
    #pragma acc parallel loop present (hist[:maxbin],gr[:maxbin])
    for (int i = 0; i < maxbin; ++i)
    {
        double nideal  = rho * ( pow((i+1)*deltaR,3) - pow(i*deltaR,3) );
        gr[i] = ((double) hist[i]) / (nideal*conf->natoms);
    }
    #pragma acc update self(gr[:maxbin], hist[:maxbin])
         
    fPtr = fopen("RDF","w");
    for (int i = 0; i < maxbin; ++i)
      fprintf(fPtr,"%lf %lf\n", i*deltaR, gr[i]);
    fclose(fPtr);
    
    free(hist);
    free(gr);
   
    free(conf);
    free_coords(coords);
    return 0;
}

In [ ]:
%%bash
cat $PWD/RDF

### Solution

Example stored in: `../../examples/C/Deep_copy_solution.c`

In [ ]:
%%idrrun -a --cliopts "0.5 15.5"
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    size_t size;
    double* data;
} Array;

typedef struct
{
    Array* x;
    Array* y;
    Array* z;
} Coordinates;

typedef struct
{
    size_t natoms;    // number of atoms
    double lx;        // box length in each dimension
    double ly;         
    double lz;         
} Config;

Array* allocate_array(size_t size);
void free_array(Array* arr);
Coordinates* read_coords(char* filepath, Config *config);
void free_coords(Coordinates* coords);

Array* allocate_array(size_t size)
{
    Array* arr = (Array*) malloc(sizeof(Array));
    arr->size = size;
    arr->data = (double*) malloc(size*sizeof(double));
    #pragma acc enter data create(arr, arr->data[:size]) copyin(arr->size)
    return arr;
}

Coordinates* allocate_coords(size_t size)
{
    Coordinates* coords = (Coordinates*) malloc(sizeof(Coordinates));
    #pragma acc enter data create(coords)
    coords->x  = allocate_array(size);
    coords->y  = allocate_array(size);
    coords->z  = allocate_array(size);
    return coords;
 }

void free_array(Array* arr)
{
    free(arr->data);
    free(arr);
    #pragma acc exit data delete(arr->data,arr)
}

void free_coords(Coordinates* coords)
{
    free_array(coords->x);
    free_array(coords->y);
    free_array(coords->z);
    #pragma acc exit data delete(coords)
    free(coords);
}

Coordinates* read_coords(char* filepath, Config* conf)
{
    FILE* fptr = fopen(filepath, "r");
    char* line = NULL;
    size_t len = 0;
    size_t i = 0;
    char n[20], x[20], y[20], z[20], vx[20], vy[20], vz[20];
    
    if (fptr == NULL)
        exit(EXIT_FAILURE);

    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->natoms = atoi(n);
    getline(&line, &len, fptr);
    sscanf(line, "%s", n);
    conf->lx = atof(n);
    conf->ly = atof(n);
    conf->lz = atof(n);
    #pragma acc enter data copyin(conf)
    printf("Number of atoms in %s file: %d\n", filepath, conf->natoms);
    
    Coordinates* coords = allocate_coords(conf->natoms);
    
    while (i < conf->natoms)
    {
        getline(&line, &len, fptr);
        sscanf(line, "%s %s %s %s %s %s %s", n, x, y, z, vx, vy, vz);
        coords->x->data[i] = atof(x);
        coords->y->data[i] = atof(y);
        coords->z->data[i] = atof(z);
	++i;
    }
    #pragma acc update device(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    fclose(fptr);
    return coords;
}

int main(int argc, char** argv)
{
    double deltaR, rCutOff;
    FILE* fPtr;
    char* input;
    double xij, yij, zij, rij;
    int d;
    
    if (argc < 3 || argc > 4) 
    {
        fprintf(stderr, "%s", "ERROR: Wrong number of parameters.\n");
        fprintf(stderr, "%s", "ERROR: The program requires at least two parameters:\n");
        fprintf(stderr, "%s", "ERROR: deltaR, the length of each bin, and \n");
        fprintf(stderr, "%s", "ERROR: rCutoff, the total length (rcut < box_length/2).\n");
        fprintf(stderr, "%s", "ERROR: Usage example: ./rdf 0.5 15.5 [input]\n");
        exit(EXIT_FAILURE);
    }
    else
    {
        deltaR = atof(argv[1]);
        rCutOff = atof(argv[2]);
        if (argc == 4)
            input = argv[3];
        else
            input = "./dyn.xyz";
    }
    
    int maxbin = rCutOff/deltaR + 1;
    int* hist = (int*) malloc(maxbin*sizeof(int));
    double* gr = (double*) malloc(maxbin*sizeof(double));
    #pragma acc enter data create(hist[:maxbin],gr[:maxbin])
    
    #pragma acc parallel loop present(hist[:maxbin])
    for (size_t i=0; i<maxbin; ++i)
        hist[i] = 0;
    
    Config* conf = (Config*) malloc(sizeof(Config));
    Coordinates* coords = read_coords(input, conf);  

    #pragma acc parallel loop present(conf,coords,coords->x,coords->y,coords->z)\
                              present(coords->x->data[:conf->natoms],coords->y->data[:conf->natoms],coords->z->data[:conf->natoms])
    for (int j = 0; j < conf->natoms; ++j)
        #pragma acc loop private(xij,yij,zij,rij,d)
        for (int i = 0; i < conf->natoms; ++i)
            if (i != j)		
            {
	        xij = coords->x->data[j]-coords->x->data[i];
                yij = coords->y->data[j]-coords->y->data[i];
                zij = coords->z->data[j]-coords->z->data[i];
                xij -= floor(xij/conf->lx + 0.5) *conf->lx;
                yij -= floor(yij/conf->ly + 0.5) *conf->ly;
                zij -= floor(zij/conf->lz + 0.5) *conf->lz;
		rij = xij*xij + yij*yij + zij*zij;
                d = (int) (sqrt(rij)/deltaR);
                if (d < maxbin)
                    #pragma acc atomic update
                    ++hist[d];
        }
    
    double rho = ((double) conf->natoms) / ((double)(conf->lx * conf->ly * conf->lz)) * 4.0 / 3.0 * acos(-1.0);
    #pragma acc parallel loop present (hist[:maxbin],gr[:maxbin])
    for (int i = 0; i < maxbin; ++i)
    {
        double nideal  = rho * ( pow((i+1)*deltaR,3) - pow(i*deltaR,3) );
        gr[i] = ((double) hist[i]) / (nideal*conf->natoms);
    }
    #pragma acc update self(gr[:maxbin])
         
    fPtr = fopen("RDF","w");
    for (int i = 0; i < maxbin; ++i)
      fprintf(fPtr,"%lf %lf\n", i*deltaR, gr[i]);
    fclose(fPtr);
    #pragma acc exit data delete(hist,gr)
    free(hist);
    free(gr);
    #pragma acc exit data delete(conf)
    free(conf);
    free_coords(coords);
    return 0;
}

In [ ]:
%%bash
cat $PWD/RDF

## Deep copy with manual attachment

It is also possible to proceed to a bottom-up deep copy, in which you can first copy sub-objects on the accelerator and then attach them to existing children.
With this procedure you will have to attach explicitly the pointers to the children. This can be easily apprehend with the subsequent code.

```C
typedef class{
    float *vx, *vy, *vz;
}velocity;

...
velocity U;
...

#pragma acc enter data copy(U.vx[0:size], U.vy[0:size], U.vz[0:size])
#pragma acc enter data copy(U)
```

Here the first copyin will pass the arrays on the device memory and the second will provide the complex datatype. But the pointers of the complex datatype (such as v0.x) will still reference the host structure.
We should thus provide to the compiler the information of the datatype as it should be on the device. This is done by adding an `attach` directive.

```C
typedef class{
    float *vx, *vy, *vz;
}velocity;

...
velocity U;
...

#pragma acc enter data copy(U.vx[0:size], U.vy[0:size], U.vz[0:size])
#pragma acc enter data copy(U) attach(U.vx, U.vy, U.vz)
```

Here, the pointer and its target are present on the device. The directive will inform the compiler to replace the host pointer with the corresponding device pointer in device memory.

The `detach` clause can be use to free the structure memory but is not mandatory.

```fortran
#pragma acc exit data detach (U.vx, U.vy, U.vz)    // not required
#pragma acc exit data copyout(U.vx, U.vy, U.vz)
#pragma acc exit data copyout(U)
```

### Exercise

In the following exercise we'll resolve the Lotka–Volterra predator–prey equations

for the prey population:

$\frac{dx}{dt}=birth_xx-death_xxy$

and for the predator population:

$\frac{dy}{dt}=birth_yxy-death_yy.$

Example stored in: `../../examples/C/Deep_copy_attach_detach_exercise.c`

In [ ]:
%%idrrun -a
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    double* count;
    double* birth_rate;
    double* death_rate;    
} Population;

void free_pop(Population* pop)
{
    // Add openacc directives
    free(pop->count);
    free(pop->birth_rate);
    free(pop->death_rate);
    free(pop);
}

void derivee(double* x, Population* pop, double* dx)
{
    // Add openacc directives
    dx[0] =  pop->birth_rate[0]*x[0] - pop->death_rate[0]*x[0]*x[1];
    dx[1] = -pop->death_rate[1]*x[1] + pop->birth_rate[1]*x[0]*x[1];       
}

void rk4(Population* pop, double dt)
{
        double* x_temp = (double*) malloc(2*sizeof(double));
        double* k1     = (double*) malloc(2*sizeof(double));
        double* k2     = (double*) malloc(2*sizeof(double));
        double* k3     = (double*) malloc(2*sizeof(double));
        double* k4     = (double*) malloc(2*sizeof(double));
        double halfdt  = dt / 2.0;
 
	// Add openacc directives
          for(int i=0; i<2; i++)
             x_temp[i] = pop->count[i];

          derivee(x_temp, pop, k1);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k1[i]*halfdt;
        
          derivee(x_temp, pop, k2);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k2[i]*halfdt;

          derivee(x_temp, pop, k3);
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k3[i]*dt;

          derivee(x_temp, pop, k4);
          for(int i=0; i<2; i++)
            pop->count[i] = pop->count[i] + (dt/6.0)*(k1[i] + 2.0*k2[i] + 2.0*k3[i] + k4[i]);
}
                 
    
int main(void)                 
{    
    double ti   =   0.00; 
    double dt   =   0.05;
    double tmax = 100.00; 
    
    Population *pred_prey = (Population *) malloc(sizeof(Population));
    pred_prey->count      = (double*) malloc(2*sizeof(double));
    pred_prey->birth_rate = (double*) malloc(2*sizeof(double));
    pred_prey->death_rate = (double*) malloc(2*sizeof(double));
    
    pred_prey->count[1]      = 15.00;  // predator count
    pred_prey->birth_rate[1] =  0.01;  // predator birth rate
    pred_prey->death_rate[1] =  1.00;  // predator death rate

    pred_prey->count[0]      = 100.00; // prey count
    pred_prey->birth_rate[0] =   2.00; // prey birth rate 
    pred_prey->death_rate[0] =   0.02; // prey death rate

    FILE* fichier = fopen("output", "w");
    // Add openacc directives
    while (ti < tmax)
    {
        ti += dt;
        rk4(pred_prey, dt);
        for(int i=0; i<2; i++)
        {
	    // Add openacc directives
	}
        fprintf(fichier, "%lf %s %lf %s %lf\n", ti,";", pred_prey->count[0],";",pred_prey->count[1]);
    }   
    fclose(fichier);
    free_pop(pred_prey);
    return 0;
}

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

data = np.genfromtxt("output", delimiter=';')
time = data[:, 0]
preys = data[:, 1]
predators = data[:, 2]

plt.plot(time, preys, color = 'blue')
plt.plot(time, predators, color = 'red')

### Solution

Example stored in: `../../examples/C/Deep_copy_attach_detach_solution.c`

In [ ]:
%%idrrun -a
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <float.h>

typedef struct
{
    double* count;
    double* birth_rate;
    double* death_rate;    
} Population;

void free_pop(Population* pop)
{
    #pragma acc exit data detach(pop->count, pop->birth_rate, pop->death_rate)
    #pragma acc exit data delete(pop->count, pop->birth_rate, pop->death_rate)    
    #pragma acc exit data delete(pop)
    free(pop->count);
    free(pop->birth_rate);
    free(pop->death_rate);
    free(pop);
}

void derivee(double* x, Population* pop, double* dx)
{
    #pragma acc serial present(pop, pop->birth_rate[0:2], pop->death_rate[0:2], x[0:2], dx[0:2])
    {
    dx[0] =  pop->birth_rate[0]*x[0] - pop->death_rate[0]*x[0]*x[1];
    dx[1] = -pop->death_rate[1]*x[1] + pop->birth_rate[1]*x[0]*x[1];       
    }
}

void rk4(Population* pop, double dt)
{
        double* x_temp = (double*) malloc(2*sizeof(double));
        double* k1     = (double*) malloc(2*sizeof(double));
        double* k2     = (double*) malloc(2*sizeof(double));
        double* k3     = (double*) malloc(2*sizeof(double));
        double* k4     = (double*) malloc(2*sizeof(double));
        double halfdt  = dt / 2.0;
 
        # pragma acc data create(k1[0:2], k2[0:2], k3[0:2], k4[0:2], x_temp[0:2]) present(pop, pop->birth_rate[0:2], pop->death_rate[0:2], pop->count[0:2])
        {

          #pragma acc parallel loop
          for(int i=0; i<2; i++)
             x_temp[i] = pop->count[i];

          derivee(x_temp, pop, k1);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k1[i]*halfdt;
        
          derivee(x_temp, pop, k2);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k2[i]*halfdt;

          derivee(x_temp, pop, k3);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            x_temp[i] = pop->count[i] + k3[i]*dt;

          derivee(x_temp, pop, k4);
          #pragma acc parallel loop
          for(int i=0; i<2; i++)
            pop->count[i] = pop->count[i] + (dt/6.0)*(k1[i] + 2.0*k2[i] + 2.0*k3[i] + k4[i]);
        }
}
                 
    
int main(void)                 
{    
    double ti   =   0.00; 
    double dt   =   0.05;
    double tmax = 100.00; 
    
    Population *pred_prey = (Population *) malloc(sizeof(Population));
    pred_prey->count      = (double*) malloc(2*sizeof(double));
    pred_prey->birth_rate = (double*) malloc(2*sizeof(double));
    pred_prey->death_rate = (double*) malloc(2*sizeof(double));
    
    pred_prey->count[1]      = 15.00;  // predator count
    pred_prey->birth_rate[1] =  0.01;  // predator birth rate
    pred_prey->death_rate[1] =   1.0;  // predator death rate

    pred_prey->count[0]      = 100.00; //prey count
    pred_prey->birth_rate[0] =   2.00; // prey birth rate 
    pred_prey->death_rate[0] =   0.02; // prey death rate

    #pragma acc enter data copyin(pred_prey->count[0:2], pred_prey->birth_rate[0:2], pred_prey->death_rate[0:2])    
    #pragma acc enter data copyin(pred_prey) attach(pred_prey->count, pred_prey->birth_rate, pred_prey->death_rate)

    FILE* fichier = fopen("output_solution", "w");
    while (ti < tmax)
    {
        ti += dt;
        rk4(pred_prey, dt);
        for(int i=0; i<2; i++)
        {
            #pragma acc update self(pred_prey->count[i:1])
	}
        fprintf(fichier, "%lf %s %lf %s %lf\n", ti, ";", pred_prey->count[0], ";", pred_prey->count[1]);
    }   
    fclose(fichier);
    free_pop(pred_prey);    
    return 0;
}

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

data = np.genfromtxt("output_solution", delimiter=';')
time = data[:, 0]
preys = data[:, 1]
predators = data[:, 2]

plt.plot(time, preys, color = 'blue')
plt.plot(time, predators, color = 'red')